In [1]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from collections import Counter
import shutil
import os
import csv
import requests
from bs4 import BeautifulSoup
from io import StringIO
import random
import re

In [2]:
DNA_inf = pd.read_csv("success_download.csv")
inf = {}

for i in range(len(DNA_inf)):
    inf[DNA_inf.loc[i,"ID"]] =  DNA_inf.loc[i,"Uniprot ID"]

In [5]:
def check_score(matrix_name, seq):
    
    jaspar_matrix = f"3_Frequency_matrix/download_JASPAR/{matrix_name}.jaspar"
    with open(jaspar_matrix) as f_temp:
        jaspar_motifs = motifs.parse(f_temp, "jaspar")

    for motif in jaspar_motifs:
        motif.__dict__['counts']
    
    score = 0
    for q in range(len(seq)):
        score += motif.__dict__['counts'][seq[q]][q]
    length = len(motif.consensus)
    
    score_max = 0
    for i in range(len(motif.consensus)):
        score_max += motif.__dict__['counts'][motif.consensus[i]][i]
    
    re = score/score_max
    return re

In [ ]:
csv_inf = []
for matrix_name in list(inf.keys()):
    pos_fasta = f"4_Positive_data/{matrix_name}_final_cleaning.fasta"
    with open(pos_fasta, "r") as file_pos:
        sequences_pos = list(SeqIO.parse(file_pos, "fasta"))
    if len(inf[matrix_name]) > 6:
        continue   
        
    for sub_pos in sequences_pos:
        csv_inf.append((matrix_name,
                       inf[matrix_name],
                       str(sub_pos.seq),
                       #int(re.search(r'\((\d+)\)', sub_pos.description).group(1))
                       check_score(matrix_name, str(sub_pos.seq)),
                        1))
        
        
    neg_fasta = f"FASTA_80hom_negative_m/{matrix_name}_matrix_similarity.fasta"
    with open(neg_fasta, "r") as file_neg:
        sequences_neg = list(SeqIO.parse(file_neg, "fasta"))
        
    for sub_neg in sequences_neg:
        
        csv_inf.append((matrix_name,
                       inf[matrix_name],
                       str(sub_neg.seq),
                       check_score(matrix_name, str(sub_neg.seq)),
                       0))
        
    print(matrix_name)    

In [7]:
# store the above data to .csv

file_path = "matrix_pos_neg_seq_80hom.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Matrix_ID", "Uniprot_ID", "DNA_Sequence","score/max_score","Label"]) 
    
    for item in csv_inf:
        writer.writerow(item)